In [1]:
from TextNormalizer import TextNormalizer
from flashtext import KeywordProcessor
import re
import pickle

## Загрузка файлов

In [5]:
def save_bin(fname,var):
    with open(fname, 'wb') as fp:
        pickle.dump(var, fp)

In [57]:
def load_bin(fname):
    with open(fname, 'rb') as fp:
        return pickle.load(fp)

In [65]:
def load_txt_doc(fname, doc_delim='<h1>'):
    with open(fname, 'r', encoding="utf-8") as fp:
        return fp.read().split(doc_delim)[1:]

In [56]:
def load_txt(fname, vec_delim = '/', phrase_delim='\n'):
    with open(fname, encoding="utf-8") as f:
        return [list(filter(lambda x: x!='',elem.split(phrase_delim))) for elem in f.read().split(vec_delim)]

## Нормализация фраз и документов

In [6]:
def normalize_vecs(vecs):
    return [[' '.join(TextNormalizer.normalize(phrase)) for phrase in vec] for vec in vecs]

In [7]:
def normalize_docs(docs):
    return [' '.join(TextNormalizer.normalize(doc)) for doc in docs]

In [77]:
# список исходных текстов и фраз
docs = load_txt_doc('docs.txt')
vecs = load_txt("vecs.txt")

In [78]:
docs

["'бытовые кондиционеры',\n  'вентилятор',\n  'домашние кондиционеры',\n  'инвертор кондиционер',\n  'инверторные кондиционеры',\n  'инверторные сплит системы',\n  'инверторный кондиционер',\n  'как самому установить кондиционер',\n  'канального типа',\n  'канальный кондиционер',\n  'кассетные кондиционеры',\n  'кассетный кондиционер',\n  'колонные кондиционеры',\n  'комнатные кондиционеры',\n  'кондиционер 12',\n  'кондиционер samsung',\n  'кондиционер акция',\n  'кондиционер без воздуховода',\n  'кондиционер для дома',\n  'кондиционер киев купить',\n  'кондиционер киев цена',\n  'кондиционер цена',\n  'кондиционеры daikin',\n  'кондиционеры fujitsu',\n  'кондиционеры gree',\n  'кондиционеры hitachi',\n  'кондиционеры lg',\n  'кондиционеры mitsubishi',\n  'кондиционеры panasonic',\n  'кондиционеры samsung',\n  'кондиционеры акция',\n  'кондиционеры воронеж',\n  'кондиционеры дайкин',\n  'кондиционеры дешево',\n  'кондиционеры для дома',\n  'кондиционеры лджи',\n  'кондиционеры лессар'

In [79]:
# делим на предложения тексты и нормализуем их
sdocs = [[' '.join(TextNormalizer.normalize(sent)) for sent in re.split('\. |! |\?',doc)] for doc in docs]

In [69]:
# список групп нормализованных ключевых фраз и список нормализованных текстов
nvecs = load_bin('nvecs')
ndocs = load_bin('ndocs')

## Решение

- делим фразы на слова
- добавляем слова в движок flashtext
- извлекаем слова из прекдложения
- добавляем фразы целиком в движок flashtext
- извлекаем фразы из полученных выше наборов ключевых слов
- возвращаем фразу если она содержится в предложении

In [70]:
def locate_phrase(phrase, clean_phrase, sentence):
    ph_words = phrase.split()
    keyword_processor = KeywordProcessor()
    for word in ph_words:
        keyword_processor.add_keyword(word)
    found_words = ' '.join(keyword_processor.extract_keywords(sentence))
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keyword(phrase, clean_phrase)
    for elem in found_words:
        found = keyword_processor.extract_keywords(found_words)
        if found:
            return found
    return []

- если вернулась фраза добавляем ее в список
- соответствие текста ключевым фразам = $\frac{|N_{found}|}{|N_{block}|}$
- недостающие фразы - невошедшие в текст (разность множеств)

In [71]:
def relevance(vec, clean_vec, doc):
    found_phrases = []
    for phrase, clean_phrase in zip(vec, clean_vec):
        for sent in doc:
            found = locate_phrase(phrase, clean_phrase, sent)
            if found:
                found_phrases.append(found[0])
                break
    relevance = len(found_phrases)/len(vec)
    missing_phrases = set(clean_vec).difference(set(found_phrases))
    return relevance, missing_phrases

In [80]:
output = relevance(nvecs[0], vecs[0], sdocs[0])

In [74]:
relevance_out = []
missing_phrases_out = []
for idoc, doc in enumerate(sdocs):
    i=1
    for vec, clean_vec in zip(nvecs, vecs):
        rel, mis = relevance(vec, clean_vec, doc)
        #relevance_out.append(rel)
        #missing_phrases_out.append(mis)
        print("\n\nСоответствие ", idoc+1, " документа к ", i, " блоку: ", rel)
        print("Возможно добавить фразы: \n", mis) 
        i+=1



Соответствие  1  документа к  1  блоку:  0.345679012345679
Возможно добавить фразы: 
 {'настенные кондиционеры', 'купить и установить кондиционер', 'сплит система lg', 'монтаж кондиционера цены', 'купить кондиционер с установкой москва', 'сплит системы волгоград', 'сплит система купить', 'кондиционеры официальный сайт', 'как самому установить кондиционер', 'домашние кондиционеры', 'купить инверторный кондиционер', 'типы кондиционеров', 'кондиционеры lg', 'кондиционеры цены', 'установка кондиционеров в москве недорого', 'кондиционер цена', 'ремонт сплит систем', 'сплит системы в краснодаре', 'кондиционеры самсунг', 'системы кондиционирования', 'купить кондиционер в интернете', 'сплит система с установкой', 'кондиционеры panasonic', 'купить кондиционер цены', 'принцип работы кондиционера', 'мобильный кондиционер купить', 'спрей для кондиционера', 'кондиционеры панасоник', 'кондиционеры воронеж', 'кондиционеры daikin', 'инверторные сплит системы', 'монтаж кондиционеров в москве', 'конди



Соответствие  2  документа к  2  блоку:  0.3
Возможно добавить фразы: 
 {'срочный ремонт кондиционеров', 'расценки на обслуживание кондиционеров', 'обслуживание кондиционера lg', 'обслуживание кондиционеров panasonic', 'прайс на обслуживание кондиционеров', 'обслуживание бытовых кондиционеров', 'ремонт и обслуживание кондиционеров', 'стоимость обслуживания кондиционеров', 'ремонт и установка кондиционеров', 'чистка кондиционеров', 'техобслуживание кондиционеров', 'сервисное обслуживание кондиционеров цена', 'разовое обслуживание кондиционеров', 'обслуживание кондиционеров daikin', 'чистка кондиционера', 'ремонт бытовых кондиционеров', 'ремонт кондиционеров цена', 'обслуживание кондиционеров mitsubishi', 'обслуживание кондиционера цена', 'техническое обслуживание кондиционера цена', 'обслуживание кондиционеров работа'}


Соответствие  2  документа к  3  блоку:  0.0
Возможно добавить фразы: 
 {'передвижной кондиционер', 'переносной кондиционер без воздуховода', 'бытовые мобильные конди



Соответствие  4  документа к  1  блоку:  0.09259259259259259
Возможно добавить фразы: 
 {'настенные кондиционеры', 'купить и установить кондиционер', 'кондиционер днепропетровск', 'сплит система lg', 'монтаж кондиционера цены', 'купить кондиционер с установкой москва', 'сплит система купить', 'сплит системы волгоград', 'кондиционеры официальный сайт', 'установить сплит систему', 'домашние кондиционеры', 'как самому установить кондиционер', 'купить инверторный кондиционер', 'типы кондиционеров', 'интернет магазин кондиционеров', 'кондиционеры lg', 'кондиционеры цены', 'установка кондиционеров в москве недорого', 'кондиционер цена', 'ремонт сплит систем', 'сплит системы в краснодаре', 'кондиционеры самсунг', 'демонтаж кондиционеров', 'напольный мобильный', 'канальная сплит система', 'чистка кондиционеров', 'купить кондиционер в интернете', 'системы кондиционирования', 'сплит система с установкой', 'кондиционеры panasonic', 'купить кондиционер цены', 'принцип работы кондиционера', 'моби

In [73]:
output

(0.345679012345679,
 {'lg сплит',
  'бытовые кондиционеры',
  'вентилятор',
  'домашние кондиционеры',
  'инвертор кондиционер',
  'инверторные кондиционеры',
  'инверторные сплит системы',
  'инверторный кондиционер',
  'как самому установить кондиционер',
  'канального типа',
  'канальный кондиционер',
  'кассетные кондиционеры',
  'кассетный кондиционер',
  'колонные кондиционеры',
  'комнатные кондиционеры',
  'кондиционер 12',
  'кондиционер samsung',
  'кондиционер акция',
  'кондиционер без воздуховода',
  'кондиционер для дома',
  'кондиционер киев купить',
  'кондиционер киев цена',
  'кондиционер цена',
  'кондиционеры daikin',
  'кондиционеры fujitsu',
  'кондиционеры gree',
  'кондиционеры hitachi',
  'кондиционеры lg',
  'кондиционеры mitsubishi',
  'кондиционеры panasonic',
  'кондиционеры samsung',
  'кондиционеры акция',
  'кондиционеры воронеж',
  'кондиционеры дайкин',
  'кондиционеры дешево',
  'кондиционеры для дома',
  'кондиционеры лджи',
  'кондиционеры лессар',


In [76]:
old_output = output[0]
old_output

0.345679012345679

In [81]:
output[0]

0.6481481481481481

### Tf-idf

In [17]:
nvecs = [TextNormalizer.normalize(text) for text in vecs]

In [18]:
ndocs = [TextNormalizer.normalize(doc) for doc in docs]

In [19]:
dct = Dictionary(nvecs)

In [20]:
vec_corpus = [dct.doc2bow(vec) for vec in nvecs]

In [21]:
doc_corpus = [dct.doc2bow(doc) for doc in ndocs]

In [23]:
model = TfidfModel(vec_corpus,dictionary=dct)

In [24]:
tf_vecs = [model[vec] for vec in vec_corpus]

In [25]:
tf_docs = [model[doc] for doc in doc_corpus]

In [26]:
index = similarities.SparseMatrixSimilarity(tf_vecs, num_features=dct.num_pos)

In [36]:
index[tf_docs[3]]

array([0.07533741, 0.06170711, 0.10711007, 0.78122646], dtype=float32)